In [1]:
from math import sqrt

In [2]:
import numpy as np

In [3]:
from geometry import triangulate, transform, wall_pieces

In [4]:
from helpers import make_mesh, write_mesh

In [5]:
EIGHTH = [
    [(0, 0, 2), (0, 1, 1), (1, 1, 1)],
    [(0, 2, 1), (0, 1, 1), (1, 1, 1)],
    [(0, 2, 1), (0, 2, 0), (1, 1, 1)],
    [(2, 2, 0), (0, 2, 0), (1, 1, 1)],
]   

In [6]:
Z_SCALE = sqrt(2)

In [7]:
WALL = 0.4
FLOOR = 0.6

In [8]:
INNER_EIGHTH = transform(EIGHTH, lambda x, y, z: (x, y, Z_SCALE * (2 - z)))

In [9]:
QUARTER_FRONT_TRI_RAW = [
    [(0, 1, 1), (0, 0, 2), (0, 1, 2)],
    [(0, 1, 1), (0, 2, 2), (0, 1, 2)],
    [(0, 1, 1), (0, 2, 1), (0, 2, 2)],
]

QUARTER_FRONT_TRI = transform(QUARTER_FRONT_TRI_RAW, lambda x, y, z: (x, y, Z_SCALE * (2 - z)))

In [16]:
def make_5_piece(eighth, front_panel, wall, floor):
    quarter = transform(eighth, lambda x, y, z: (x, y, z + floor)) + transform(eighth, lambda x, y, z: (y, x, z + floor))

    width = max(p[0] for tri in eighth for p in tri)
    front_bottom_rect = [
        (0, 0, 0), (0, 0, floor), (0, width, floor), (0, width, 0)
    ]
    front = transform(front_panel, lambda x, y, z: (x, y, z + floor)) + list(triangulate(front_bottom_rect))

    mould_quarter = quarter + front

    shape_height = max(p[2] for tri in eighth for p in tri)
    height = shape_height + floor
    size = width + wall
    intermediate_heights = [p[2] for tri in front_panel for p in tri if p[1] == 0 and p[0] == width]
    heights = [0, floor] + [x + floor for x in intermediate_heights] + [height]

    rim_quarter = list(triangulate(
        [(0, width, height), (0, size, height), (size, size, height), (width, width, height)]
    ))

    top_rim = (
        rim_quarter
        + transform(rim_quarter, lambda x, y, z: (y, x, z))
        + transform(rim_quarter, lambda x, y, z: (x, -y, z))
        + transform(rim_quarter, lambda x, y, z: (y, -x, z))
    )

    floor_square = list(triangulate(
        [(0, 0, 0), (0, width, 0), (width, width, 0), (width, 0, 0)]
    ))
    floor_rim = transform(top_rim, lambda x, y, z: (x, y, 0))
    floor_all = floor_rim + floor_square

    left_outer_pieces = [
        [(0, width), (0, size)],
        [(0, size), (size, size)],
        [(size, size), (size, 0)],
    ]

    outer_pieces = left_outer_pieces + [[(x, -y) for x, y in piece] for piece in left_outer_pieces]
    wall_outer = list(tri for piece in outer_pieces for tri in wall_pieces(*piece, heights=heights))

    wrapper = floor_all + wall_outer + top_rim
    outer_mould = mould_quarter + wrapper
    division = (
        transform(front, lambda x, y, z: (y, x, z))
        + list(wall_pieces((width, 0), (width, -width), heights))
        + list(wall_pieces((width, -width), (0, -width), heights))
    )

    filler_square = transform(floor_square, lambda x, y, z: (x, -y, z))
    filler_mould_quarter = transform(mould_quarter, lambda x, y, z: (x, -y, z))
    filler = filler_square + filler_mould_quarter

    piece_1 = outer_mould + filler
    piece_4 = outer_mould + division
    piece_5 = filler + division
    
    return piece_1, piece_4, piece_5

In [17]:
PIECE_1, PIECE_4, PIECE_5 = make_5_piece(INNER_EIGHTH, QUARTER_FRONT_TRI, WALL, FLOOR)

In [18]:
MM_SCALE = 15

In [19]:
write_mesh(PIECE_1, MM_SCALE, 'output/piece_1.stl')

In [20]:
write_mesh(PIECE_4, MM_SCALE, 'output/piece_4.stl')

In [21]:
write_mesh(PIECE_5, MM_SCALE, 'output/piece_5.stl')